In [1]:
include("../LiPoSID.jl")

using LinearAlgebra

using DynamicPolynomials

using SymPy

using Plots

using Dates

using HDF5

data_dir = "../DATA/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

b⁰ = [0, 0, 1, 1]
b¹ = [0, 0, -1, 1]
bˣ = [1, 0, 0, 1]
bʸ = [1, 1, 0, 1]

function read_as_operators(models_file_name, γ)
    h5open(models_file_name, "r") do file
        w = read(file["gamma_"*string(γ)]["w"])
        H = [ w        0
              0        0   ]
        
        g = read(file["gamma_"*string(γ)]["g"])
        J = [ 0     g
              0     0. + 0im  ]

        return H, J
    end
end


read_as_operators (generic function with 1 method)

In [2]:
using QuantumOptics
basis = NLevelBasis(2)

test_files = ["State_D"*string(n) for n=1:20];

#models_file_name= "../MODELS/DMD_TS_POP_w&g_bx_2024-Jan-02_at_15-50.h5"
models_file_name= "../MODELS/DMD_TS_POP_w&g_bx_2024-Jan-05_at_13-18.h5"

"../MODELS/DMD_TS_POP_w&g_bx_2024-Jan-05_at_13-18.h5"

In [3]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_dir = "../TESTS/"
tests_data_file_name = "DMD_TS_POP_testD20_"*date_and_time_string * ".h5"

for γᵢ in γ_list

    println()

    println("γ="*γᵢ)

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 


    for df in test_files # loop over initial states
        
        print(df*" - ")

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)

        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        t = convert(Vector{Float64}, tₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        Hˢⁱᵈₗₘₑ, Jˢⁱᵈₗₘₑ = read_as_operators(models_file_name, γᵢ)

        tˢⁱᵈₗₘ, ρˢⁱᵈₗₘₑ  = timeevolution.master(t, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑ), [DenseOperator(basis,Jˢⁱᵈₗₘₑ)])

        Fˢⁱᵈₗₘₑ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈₗₘₑ)]

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group
            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_sid_lme"] = convert.(Float64, Fˢⁱᵈₗₘₑ)
        end
                    
        print(minimum(Fˢⁱᵈₗₘₑ))
        print(" | ")
    
    end

end

println()
println(tests_data_file_name)
    


γ=0.079477
State_D1 - 

0.5853004845887302 | State_D2 - 0.5869762549728661 | State_D3 - 

0.5851360603474203 | State_D4 - 0.5867329417704119 | State_D5 - 0.37499265606427523 | State_D6 - 0.37499265606427523 | State_D7 - 

0.12022542667240732 | State_D8 - 0.1279163662800505 | State_D9 - 0.9352288420775875 | State_D10 - 0.9349953120913662 | State_D11 - 

0.5851360603474203 | State_D12 - 0.5867329417704119 | State_D13 - 0.5853004845887302 | State_D14 - 0.5869762549728661 | State_D15 - 

...

0.9900201729034374 | 
γ=79.477
State_D1 - 0.9927836704837226 | State_D2 - 0.9935276946531189 | State_D3 - 0.9946137075574804 | State_D4 - 0.995368032489007 | State_D5 - 0.9941648309578185 | State_D6 - 0.994164830957595

 | State_D7 - 0.9926583374367831 | State_D8 - 0.9933977986846152 | State_D9 - 0.9929989362842954 | State_D10 - 0.9959032331052838 | State_D11 - 0.9946137075577658 | State_D12 - 0.9953680324889118 | State_D13 - 

0.9927836704843016 | State_D14 - 0.9935276946533451 | State_D15 - 0.9952972880841 | State_D16 - 0.9952972880837065 | State_D17 - 0.9926583374363384 | State_D18 - 0.9933977986845642 | State_D19 - 

0.9959032331052977 | State_D20 - 0.9929989362840564 | 
γ=251.33
State_D1 - 0.9964741233015956 | State_D2 - 0.9951722981661939 | State_D3 - 0.995723809220254 | State_D4 - 0.9944921686791255 | State_D5 - 0.9934381011222587 | State_D6 - 0.9934381011222587 | State_D7 - 0.9993930049944176 | State_D8 - 0.9983585387881854 | State_D9 - 0.9947956521843309 | State_D10 - 

0.9936367248006006 | State_D11 - 0.9957238092202408 | State_D12 - 0.9944921686791255 | State_D13 - 0.9964741233015946 | State_D14 - 0.9951722981662566 | State_D15 - 0.9929931372349909 | State_D16 - 0.9929931372349436 | State_D17 - 0.9993930049944258 | State_D18 - 0.9983585387881801 | State_D19 - 0.9936367248006006 | State_D20 - 0.994795652184223 | DMD_TS_POP_testD20_2024-Jan-05_at_13-24.h5
